In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset/03262023_CNN.csv')

In [3]:
df.head()

,link,title,article,date,content
0,https://www.cnn.com/2023/03/26/us/delaware-riv...,Philadelphia officials say drinking water rema...,us,2023-03-26,The City of Philadelphia says it is now confid...
1,https://www.cnn.com/2023/03/25/us/west-reading...,Death toll from explosion at Pennsylvania cand...,us,2023-03-26,A fourth person has been confirmed dead in an ...
2,https://www.cnn.com/2023/03/25/us/jonathan-maj...,Actor Jonathan Majors is arrested on assault c...,us,2023-03-26,"Actor Jonathan Majors, who has recently starre..."
3,https://www.cnn.com/2023/03/26/us/georgia-weat...,2 tigers recaptured after escaping Georgia saf...,us,2023-03-26,Two tigers have been recaptured after escaping...
4,https://www.cnn.com/2023/03/26/us/indiana-univ...,12 injured in floor collapse at off-campus apa...,us,2023-03-26,Twelve people were injured Saturday night when...


In [4]:
df.content.fillna('no content', inplace=True)

In [5]:
df.isnull().sum()

link       0
title      0
article    0
date       0
content    0
dtype: int64

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
def preprocess(sentence):
    words = nltk.word_tokenize(sentence)
    words = [stemmer.stem(word) for word in words if word not in stopwords and word.isalnum()]
    return ' '.join(words)

In [9]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.PorterStemmer()
vectorizer = TfidfVectorizer()

In [10]:
def summarize_text(text):
    sentences = nltk.sent_tokenize(text.lower())
    preprocessed_text = [preprocess(sentence) for sentence in sentences]
    vectorized_text = vectorizer.fit_transform(preprocessed_text)
    lsa = TruncatedSVD(n_components=2)
    lsa_text = lsa.fit_transform(vectorized_text)
    similarity_matrix = cosine_similarity(lsa_text)
    ranked_sentences = sorted(((score, index) for index, score in enumerate(similarity_matrix[0])), reverse=True)
    top_sentences = [sentences[index] for score, index in ranked_sentences[:2]]
    summary = ' '.join(top_sentences)
    return summary

In [11]:
df['link'][43]

'https://www.cbs58.com/news/150-tenants-evacuated-from-milwaukee-complex-after-health-department-detects-toxic-levels-of-chemical'

In [12]:
df['content'][43]

'no content'

In [13]:
summarize_text_list = []
for i in range(df.shape[0]):
    if df['content'][i] == 'no content':
        summarize_text_list.append('no content')
    else:
        sum_text =summarize_text(df['content'][i]) 
        summarize_text_list.append(sum_text)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
df.shape, len(summarize_text_list)

((528, 5), 528)

In [15]:
df['Summarized_Content_LSA'] = summarize_text_list

In [16]:
df.to_csv('dataset/03262023_CNN_LSA.csv', index=False)